In [62]:
import json

In [121]:
import pandas as pd
pd.set_option('display.max_rows', None)

# Assemble Dataset

In [65]:
master = pd.read_csv('Pokemon Database.csv', encoding='latin-1')
galar = pd.read_csv('pokemon-swordshield.csv')
ioa = pd.read_csv('ioa.csv')
with open('smogon.json') as f:
    smogon = json.load(f)

In [85]:
smogondf = pd.DataFrame(smogon['pokemon'])[['name', 'formats', 'oob']]

In [86]:
galar_names = galar['name01']

In [105]:
smogondf['nname'] = [d.split('-')[0] if len(d.split('-')) != 1 else d for d in smogondf['name']]
smogondf = smogondf.groupby('nname').agg({'formats': 'sum'})

In [21]:
ioa_names = ioa['pok']

In [30]:
swsh_names = set(galar_names.append(ioa_names))

In [106]:
swsh = master[master['Pokemon Name'].isin(swsh_names)]

In [107]:
swsh = swsh[~swsh['Alternate Form Name'].isin(['Mega', 'Mega X', 'Mega Y'])]
swsh = swsh.merge(smogondf, how='left', left_on='Pokemon Name', right_on='nname')

In [108]:
swsh.to_csv('pokemon_swsh_ioa.csv', index=False)

# Parse Types

In [109]:
swsh = pd.read_csv('pokemon_swsh_ioa.csv')[['Pokedex Number', 'Pokemon Name', 'Primary Type', 'Secondary Type', 'formats']]

In [110]:
types = ['Normal','Fire','Fighting','Water','Flying','Grass','Poison','Electric','Ground','Psychic','Rock','Ice','Bug','Dragon','Ghost','Dark','Steel','Fairy']

In [116]:
counts, pokemon = {}, {}
for t in types:
    sub = swsh[(swsh['Primary Type'] == t) | (swsh['Secondary Type'] == t)].drop_duplicates(subset=['Pokemon Name'])
    pokemon[t] = sub
    counts[t] = len(sub)
counts = {k: v for k, v in sorted(counts.items(), key=lambda item: item[1])}

In [117]:
counts

{'Rock': 25,
 'Fire': 29,
 'Electric': 32,
 'Ice': 32,
 'Dragon': 32,
 'Fighting': 37,
 'Steel': 37,
 'Poison': 39,
 'Ghost': 39,
 'Fairy': 41,
 'Flying': 43,
 'Dark': 43,
 'Bug': 44,
 'Ground': 48,
 'Psychic': 52,
 'Grass': 56,
 'Normal': 57,
 'Water': 87}

In [122]:
pokemon['Water']

,Pokedex Number,Pokemon Name,Primary Type,Secondary Type,formats
6,7,Squirtle,Water,NaN,['LC']
7,8,Wartortle,Water,NaN,['NFE']
8,9,Blastoise,Water,NaN,"['RU', 'National Dex', 'Uber']"
23,892,Urshifu,Fighting,Water,"['OU', 'OU', 'Uber', 'Uber']"
57,54,Psyduck,Water,NaN,['LC']
71,116,Horsea,Water,NaN,['LC']
72,117,Seadra,Water,NaN,['NFE']
73,118,Goldeen,Water,NaN,['LC']
74,119,Seaking,Water,NaN,['Untiered']
75,120,Staryu,Water,NaN,['LC']
